In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#! pip install git+https://github.com/beroguedou/SpecAugment.git
#!pip install torchaudio

In [3]:
import os
import csv
import time
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
from decode import *
import torch
import torchaudio
import numpy as np
import torch.optim as optim

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
%%time 

limit = 80
params = {'batch_size': 20,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}


#tokenizer = tokenizer_librispeech(limit=800)
training_set = LibriSpeechDataset(limit=limit, n_frames=600, version='train-clean-360')
dev_set = LibriSpeechDataset(limit=limit, n_frames=600, version='dev-clean')

train_dataloader = data.DataLoader(training_set, **params)
dev_dataloader = data.DataLoader(dev_set, **params)

steps_per_epoch = len(training_set) // params['batch_size']

#vocab_tar_size = len(tokenizer.word_index)
embedding_dim_target = 100

CPU times: user 326 ms, sys: 52.6 ms, total: 379 ms
Wall time: 1.07 s


In [5]:
#tknz = modern_tokenizers()

LibriSpeechDataset(limit=limit, n_frames=600, version='train-clean-360')[0]

(tensor([[[-1.4125, -1.2459, -1.0972,  ...,  0.0000,  0.0000,  0.0000],
          [-0.6660, -0.7965, -0.8682,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0153, -0.2704, -0.4284,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.1286, -0.2636, -0.4296,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0151,  0.3055, -0.0952,  ...,  0.0000,  0.0000,  0.0000],
          [-0.7348, -0.3408, -0.2402,  ...,  0.0000,  0.0000,  0.0000]]]),
 tensor([  101,  2045,  2001,  2625,  1997,  1037,  4306,  2055,  2085,  1998,
         14163,  6894,  5289,  2106,  2025,  2031,  2000,  2562,  2067,  1037,
          5481,  2004,  2002,  2441,  1996,  9445,  3460,  6031,  2988,  1996,
         14460,  4214,  8721,  2000, 10767,  1998, 27468,  1996,  2221,  7522]))

In [6]:
# 'bahdanau_basic'
# 'bahdanau_audio'
# 'luong_dot'
# 'luong_concat'
# 'luong_general'
# 'super_attn'

In [7]:
encoder = EncoderCONV2DRNN(batch_size=params['batch_size'], device=device, hidden_size=64).to(device)

decoder = DecoderATTRNN(30000, dec_units=64, batch_sz=params['batch_size'], 
                         hidden_size=64).to(device)


encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [ ]:
# Train the model
global_trainer(10, train_dataloader, dev_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer,
                                    criterion, device, params['batch_size'],  bad_audio_mode=True)

 ====== ====== ====== ====== ====== ======
      The model has 113250107 parameters
 ====== ====== ====== ====== ====== ======


Epoch        8: 100%|████████████████████| 4/4 [00:36<00:00,  9.19s/it, Train loss 4.9035]

In [ ]:
mfccs, references = training_set[1]
references = [tokenizer.index_word[ind] for ind in references.numpy()]
evaluate(mfccs.unsqueeze(0), references, 40, encoder, decoder, targ_lang=tokenizer, 
          device=device, beam_search=True)

In [ ]:
# Vérifications

# 1 - Data Augmentation
# 2 - Encoder
# 3 - Attention Mechanism Bahdanau Audio
# 4 - Smoothing and Topk to the attention
# 5 - Decoder 
# 6 - Métrique BLEU

In [ ]:
from transformers import BertTokenizer, BertModel
from tokenizers import BertWordPieceTokenizer

# Let's see how to increase the vocabulary of Bert model and tokenizer
trained_bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Let's train a wordpiece model tokenizer on our data 
our_bert_tokenizer = modern_tokenizers()
# Now we will collect the tokens that are'nt in pretrained bert token
length = our_bert_tokenizer.get_vocab_size()
list_new_tokens = [our_bert_tokenizer.id_to_token(id_) for id_ in range(length)\
              if our_bert_tokenizer.token_to_id(our_bert_tokenizer.id_to_token(0)) != None] 
# Adding the new tokens to the bert's one
num_added_toks = trained_bert_tokenizer.add_tokens(list_new_tokens)
print('We have added', num_added_toks, 'tokens')
# Load a bert pretrained model
bert_model = BertModel.from_pretrained('bert-base-uncased')
# Put the model in "evaluation" mode, meaning feed-forward operation.
bert_model.eval()
# Notice: resize_token_embeddings 
bert_model.resize_token_embeddings(len(trained_bert_tokenizer))
# Return tokenizer word_embeddings
trained_bert_tokenizer, bert_model.embeddings.word_embeddings



In [ ]:
tokenizer.encode('toto va chez tata')

In [ ]:


# Idea use Bert tokenizer to tokenize target data 
#  Use bert Embedding for the Embedding of the decoder 
# Verifier les poids qui s'entrainent